<a href="https://colab.research.google.com/github/Flieder13/Genre-AI/blob/main/3_genre_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import ctypes
!pip install spotipy
import spotipy
import spotipy.util as util
from enum import Enum

from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id="307be7e493184da79cb233982f38b912",
                                                      client_secret="392419168fe94ee9a057567959a67b82"))

class Genre(Enum):
  EDM = 0
  Rock = 1
  RnB = 2

def WriteToInputFile(audio_features, txtdoc, AllSongsAudio_features):
  txtdoc.write(str(audio_features['energy']) + '\n')
  txtdoc.write(str(audio_features['liveness']) + '\n')
  txtdoc.write(str(audio_features['tempo']) + '\n')
  txtdoc.write(str(audio_features['speechiness']) + '\n')
  txtdoc.write(str(audio_features['acousticness']) + '\n')
  txtdoc.write(str(audio_features['instrumentalness']) + '\n')
  txtdoc.write(str(audio_features['danceability']) + '\n')
  txtdoc.write(str(audio_features['key']) + '\n')
  txtdoc.write(str(audio_features['duration_ms']) + '\n')
  txtdoc.write(str(audio_features['loudness']) + '\n')

  if(audio_features == AllSongsAudio_features[-1]):
    txtdoc.write(str(audio_features['valence']))
  else:
    txtdoc.write(str(audio_features['valence']) + '\n')
  

def get_playlist_audio_features(username, playlist_id, sp, IsTraining):
    offset = 0
    songs = []
    items = []
    ids = []
    while True:
        content = sp.user_playlist_tracks(username, playlist_id, fields=None, limit=100, offset=offset, market=None)
        songs += content['items']
        if content['next'] is not None:
            offset += 100
        else:
            break

    for song in songs:
        ids.append(song['track']['id'])

    index = 0
    AllSongsAudio_features = []
    while index < len(ids):
        AllSongsAudio_features += sp.audio_features(ids[index:index + 50])
        index += 50

    #If the net is training open the trainingfile, if it is beeing used, open the normal use-file
    if(IsTraining)
      txtdoc = open('TrainingInputFile.txt','w')
    else
      textdoc = open('InputValueFile.txt','w')

    TargetValueFile = open('TargetValueFile.txt','w')
    Musicgenre = Genre.EDM

    for audio_features in AllSongsAudio_features:

        WriteToInputFile(audio_features, txtdoc, AllSongsAudio_features)

        if(Musicgenre == Genre.EDM):
            TargetValueFile.write('1\n0\n0\n')
            Musicgenre = Genre.Rock
          
        elif(Musicgenre == Genre.Rock):
            TargetValueFile.write('0\n1\n0\n')
            Musicgenre = Genre.RnB

        else:
            if(audio_features == AllSongsAudio_features[-1]):
              TargetValueFile.write('0\n0\n1')
            else:
              TargetValueFile.write('0\n0\n1\n')

            Musicgenre = Genre.EDM
   


get_playlist_audio_features('pj9jhvsc5nzidorj96z7ygqj7','0GKTZUcmG5pma4vvMYcKYG', sp)


#Finding the dll, which has to be in the same folder as the .py file (this is done by the "./")
Dll = ctypes.CDLL('./NeuralNetDll.dll')

#Load the weights of the Neurons and edges
Dll.LoadNet()

#Tell python wich in- and outputs the Training function has, then call it
Dll.TrainNet.argtypes = [ctypes.c_uint]
Dll.TrainNet.resttype = None
Dll.TrainNet()

#Call the function to use the net
Dll.UseNet()

#Tell python wich in- and outputs the GetResult- function has, then call it for each song genre
Dll.GetNetResult.argtypes = [ctypes.c_uint]
Dll.GetNetResult.restype = ctypes.c_double

print(Dll.GetNetResult(ctypes.c_uint(0)))
print(Dll.GetNetResult(ctypes.c_uint(1)))
print(Dll.GetNetResult(ctypes.c_uint(2)))


#Quellen: Ctypes: https://www.youtube.com/watch?v=sKtQzLM5XTw   
#3Blue1Brown über NeuraleNetze: https://www.youtube.com/watch?v=aircAruvnKk&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi
# Dll erstellen in Visual Studio: https://www.youtube.com/watch?v=dcExWVrxnHM